In [1]:
import os, json

import modelscope

In [2]:


from modelscope import snapshot_download
from modelscope.msdatasets import MsDataset
# 方式一
# ds =  MsDataset.load('iic/address-ner-ccks-2021')

# 方式二（本文采用方式）
! git clone https://www.modelscope.cn/datasets/iic/address-ner-ccks-2021.git

fatal: destination path 'address-ner-ccks-2021' already exists and is not an empty directory.


In [17]:


def extract_entities_from_bio(text, tags):
    entities = []
    entity_name = ""
    flag = []
    for char, tag in zip(text, tags):
        if tag.startswith("B-"):
            if entity_name:
                # 当前实体结束，添加到entities列表中
                entities.append({"word": entity_name, "type": flag[0]})
                entity_name = ""
            flag = [tag[2:]]  # 获取实体类型
            entity_name += char
        elif tag.startswith("I-") and entity_name:
            # 继续当前实体
            entity_name += char
            flag.append(tag[2:])
        else:
            if entity_name:
                # 当前实体结束，添加到entities列表中
                entities.append({"word": entity_name, "type": flag[0]})
                entity_name = ""
                flag = []
    if entity_name:
        # 添加最后一个实体
        entities.append({"word": entity_name, "type": flag[0]})
    return entities


def parse_bio_file(file_path):

    with open(file_path, "r", encoding="utf-8") as fin:
        data = fin.read()
    example_list = data.split("\n\n")
    # print(example_list[0])
    # print("--")
    # print(example_list[1])
    # print(len(example_list))

    for example in example_list:
        word_tag_list = example.split("\n")
        word_list, tag_list = [], []
        for word_tag in word_tag_list:
            word_tag_split = word_tag.split()
            word_list.append(word_tag_split[0])
            tag_list.append(word_tag_split[1])
        print(word_list)
        print(tag_list)
        entities = extract_entities_from_bio("".join(word_list), tag_list)
        print("entities: ", entities)
        break





parse_bio_file("address-ner-ccks-2021/dev.conll")
    






['杭', '州', '五', '洲', '国', '际']
['B-city', 'E-city', 'B-poi', 'I-poi', 'I-poi', 'E-poi']
entities:  [{'word': '杭', 'type': 'city'}, {'word': '五洲国', 'type': 'poi'}]


In [37]:
def bieo_to_text_and_entities(file_path):
    results = []  # 用于存储每个样本的文本及实体

    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    current_text = []  # 当前样本的文本
    current_entity = []  # 当前实体的内容
    current_type = None
    entities = []  # 当前样本的实体列表

    for line in lines:
        line = line.strip()
        if not line:  # 遇到空行，保存当前样本
            if current_entity:  # 保存未结束的实体
                entities.append({"type": current_type, "text": "".join(current_entity)})
            if current_text:  # 保存当前样本
                results.append({"text": "".join(current_text), "entities": entities})
            # 重置状态
            current_text = []
            current_entity = []
            current_type = None
            entities = []
            continue

        token, tag = line.split()  # 假设每行格式为 "单词 TAG"
        current_text.append(token)

        if tag.startswith("B-"):  # 新实体的开始
            if current_entity:  # 保存未结束的实体
                entities.append({"type": current_type, "text": "".join(current_entity)})
            current_entity = [token]
            current_type = tag[2:]
        elif tag.startswith("I-") and current_type == tag[2:]:  # 当前实体的中间部分
            current_entity.append(token)
        elif tag.startswith("E-") and current_type == tag[2:]:  # 当前实体的结束
            current_entity.append(token)
            entities.append({"type": current_type, "text": "".join(current_entity)})
            current_entity = []  # 重置实体
            current_type = None
        elif tag == "O":  # 非实体
            if current_entity:  # 保存未结束的实体
                entities.append({"type": current_type, "text": "".join(current_entity)})
            current_entity = []
            current_type = None

    # 文件结束后处理最后一个样本
    if current_text:
        if current_entity:  # 保存未结束的实体
            entities.append({"type": current_type, "text": "".join(current_entity)})
        results.append({"text": "".join(current_text), "entities": entities})

    return results


# 示例：假设有一个名为 "bieo_data.txt" 的 BIEO 文件
# file_path = "address-ner-ccks-2021/dev.conll"
# samples = bieo_to_text_and_entities(file_path)

# # 打印结果
# for sample in samples:
#     print(f"完整文本: {sample['text']}")
#     for entity in sample["entities"]:
#         print(f"  实体类型: {entity['type']}, 实体文本: {entity['text']}")
#     break


entity_type_list = ["Province", "city", "district", "town", "road", "road_number", "poi", "house_number", "other"]


def parse_bie_file(file_path):

    all_prompt_example_list = []

    
    samples = bieo_to_text_and_entities(file_path)
    for sample in samples:
        
        text = sample["text"]
        example_input = text
        entities = sample["entities"]
        entities = {entity["type"]: entity["text"] for entity in entities}
        
        example_output = ""
        for entity_type in entity_type_list:
            entity_text = entities.get(entity_type, None)
            if not entity_text:
                continue
            example_output += entity_type + "=" + entity_text + "\n"
        one_example = {"query": example_input, "response": example_output}
        all_prompt_example_list.append(one_example)
    
    return all_prompt_example_list  
        
all_prompt_example_list = parse_bie_file("address-ner-ccks-2021/dev.conll")


In [38]:
all_prompt_example_list[0]

{'query': '杭州五洲国际', 'response': 'city=杭州\npoi=五洲国际\n'}

In [44]:

"""
输入：浙江省杭州市余杭区五常街道文一西路969号淘宝城5号楼，放前台

输出：Province=浙江省 city=杭州市 district=余杭区 town=五常街道 road=文一西路road_number=969号 poi=淘宝城 house_number=5号楼 other=，放前台
"""
import random

prompts = [
    "请解析以下地址文本，提取出如下字段，并按顺序返回：['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']。如果某字段在文本中不存在，请返回空字符串。",
    "将地址拆分为以下要素：省、市、区、乡镇、路名、门牌号、兴趣点、楼栋号及其他信息，并按顺序输出。如果缺少某部分，使用空字符串填充。",
    "请将地址文本解析为结构化字段，顺序为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，无对应字段时返回空。",
    "根据地址内容，提取并返回省、市、区、乡镇等字段。输出顺序固定为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，无内容的字段置空。",
    "请对地址文本进行要素解析，返回['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']格式的字段列表，缺失字段留空。",
    "解析地址信息，将其分解为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，若某字段不存在，返回空字符串。",
    "请从输入的地址文本中识别出以下字段：省、市、区、乡镇、路名、门牌号等，按['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序返回结果，缺失字段填空。",
    "将地址解析为各个语义单元，字段顺序为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，并确保未识别到的字段返回空字符串。",
    "请从地址文本中提取如下字段：省、市、区等。结果按['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序返回，缺失字段置空。",
    "请对以下地址文本进行字段解析，按照['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']的顺序返回解析结果，无法找到的字段置空。",
    "将输入地址分解为省、市、区、POI 等字段，按顺序返回 ['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，缺失字段返回空字符串。",
    "请对地址文本进行分解，并将其整理为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']的格式输出，未找到的部分用空字符串填充。",
    "根据输入的地址，提取并返回省、市、区、POI 等字段，按['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序输出，缺失的内容用空字符串代替。",
    "解析地址，输出各字段：['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']。如果某字段内容缺失，返回空值。",
    "请提取地址中的关键信息，并按照['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序输出，缺失字段置空。",
    "将输入的地址文本分解为语义明确的字段，并按照['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序返回。未找到对应内容时，字段留空。",
    "解析以下地址，将其结构化为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']的字段列表，缺失部分用空字符串表示。",
    "请对输入的地址进行语义拆分，并按照['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']顺序返回结构化数据，无对应内容时留空。",
    "从地址中提取省、市、区等字段，并按照['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']输出结果，缺少内容时返回空字符串。",
    "请将地址解析为结构化字段，依次返回['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']的值，若字段缺失，返回空值。"
]


def gen_prompt_example(file_bie_path):
    
    all_prompt_example_list = parse_bie_file("address-ner-ccks-2021/dev.conll")

    

    def gen_one_prompt_example(prompt, example):
        prompt_input = ""
        prompt_input += prompt_input + "\n" + prompt + "\n输入地址：" + example["query"]
        prompt_response = example["response"]

        return {"query": prompt_input, "response": prompt_response}
        
    all_sft_prompt_data_list = []
    for example in all_prompt_example_list:
        sample_prompts = random.sample(prompts, 1)  # 此户可以根据单个样本选择多个 prompt
        for prompt in sample_prompts:
            all_sft_prompt_data_list.append(gen_one_prompt_example(prompt, example))

    with open("ner_sft_data.jsonl", "w", encoding="utf-8") as fout:
        for sft_example in all_sft_prompt_data_list:
            fout.write(json.dumps(sft_example, ensure_ascii=False) + "\n")
    
    print(all_sft_prompt_data_list[0])
    print(len(all_sft_prompt_data_list))
        
gen_prompt_example("address-ner-ccks-2021/train.conll")
    




{'query': "\n将地址解析为各个语义单元，字段顺序为['Province', 'city', 'district', 'town', 'road', 'road_number', 'poi', 'house_number', 'other']，并确保未识别到的字段返回空字符串。\n输入地址：杭州五洲国际", 'response': 'city=杭州\npoi=五洲国际\n'}
1970
